# MPPI (Model Predictive Path-Integral) Control

In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
from typing import Tuple
from matplotlib import patches
from matplotlib.animation import ArtistAnimation
from IPython import display

## Control Target : Vehicle

<img src="../media/pathtracking.png" width="500px">

In [2]:
class Vehicle():
    def __init__(
            self,
            max_ang_abs: float = 0.523,  # [rad] maximum yaw acceleration magnitude
            max_lin_abs: float = 2.000,   # [m/s^2] maximum linear acceleration magnitude
            ref_path: np.ndarray = np.array([[-100.0, 0.0], [100.0, 0.0]]),
            delta_t: float = 0.02,        # [s]
            visualize: bool = True,
        ) -> None:
        """
        Initialize vehicle environment.

        State variables (9-dim):
            [x, y, yaw, Vx, Vy, Vyaw]
        Control input (u): 
            [ax_cmd, ay_cmd, ayaw_cmd]

        Note: The dynamics use a simple double-integrator model.
        """
        # Vehicle parameters.
        self.ref_pos = (0,0)
        self.max_ang_abs = max_ang_abs  # maximum yaw acceleration (rad/s^2)
        self.max_lin_abs = max_lin_abs  # maximum linear acceleration (m/s^2)
        self.delta_t = delta_t          # time step [s]
        self.ref_path = ref_path

        # Visualization settings.
        self.vehicle_w = 3.00  # vehicle width for drawing [m]
        self.vehicle_l = 4.00  # vehicle length for drawing [m]
        self.view_x_lim_min, self.view_x_lim_max = -20.0, 20.0
        self.view_y_lim_min, self.view_y_lim_max = -25.0, 25.0

        # Reset environment.
        self.visualize_flag = visualize
        self.reset()

    def reset(
            self, 
            init_state: np.ndarray = np.zeros(6)  # default state: zeros
        ) -> None:
        """Reset environment to initial state."""
        self.state = init_state.copy()
        self.frames = []
        
        self.ref_pos = (0,0)

        if self.visualize_flag:
            # Prepare figure.
            self.fig = plt.figure(figsize=(9,9))
            self.main_ax = plt.subplot2grid((3,4), (0,0), rowspan=3, colspan=3)
            self.Mz_ax = plt.subplot2grid((3,4), (0,3))  # For yaw acceleration view.
            self.Fx_ax = plt.subplot2grid((3,4), (1,3))  # For net linear acceleration view.
            self.Fy_ax = plt.subplot2grid((3,4), (2,3))  # (Unused in this example.)

            # Graph layout settings.
            self.main_ax.set_aspect('equal')
            self.main_ax.set_xlim(self.view_x_lim_min, self.view_x_lim_max)
            self.main_ax.set_ylim(self.view_y_lim_min, self.view_y_lim_max)
            self.main_ax.tick_params(labelbottom=False, labelleft=False, labelright=False, labeltop=False)
            self.main_ax.tick_params(bottom=False, left=False, right=False, top=False)

            # Set titles for control input views.
            self.Mz_ax.set_title("Mz", fontsize="12")
            self.Mz_ax.axis('off')
            self.Fx_ax.set_title("Fx", fontsize="12")
            self.Fx_ax.axis('off')
            self.Fy_ax.set_title("Fy", fontsize="12")
            self.Fy_ax.axis('off')

            self.fig.tight_layout()

    def update(
            self, 
            u: np.ndarray, 
            delta_t: float = 0.0, 
            append_frame: bool = True, 
            optimal_traj: np.ndarray = np.empty(0),  # predicted optimal trajectory from MPPI
            sampled_traj_list: np.ndarray = np.empty(0),  # sampled trajectories from MPPI
            ref_pos = (0,0)
        ) -> None:
        """
        Update the state variables using a double-integrator model.
        
        State vector:
            [x, y, yaw, Vx, Vy, Vyaw]
        Control input (u): 
            [ax_cmd, ay_cmd, ayaw_cmd]
        """
        self.ref_pos = ref_pos
        
        # Unpack current state.
        x, y, yaw, Vx, Vy, Vyaw = self.state
        dt = self.delta_t if delta_t == 0.0 else delta_t

        # Clip the control inputs.
        ax_cmd = np.clip(u[0], -self.max_lin_abs, self.max_lin_abs)
        ay_cmd = np.clip(u[1], -self.max_lin_abs, self.max_lin_abs)
        ayaw_cmd = np.clip(u[2], -self.max_ang_abs, self.max_ang_abs)

        # Update positions and orientation.
        new_x = x + Vx * dt + 0.5 * ax_cmd * dt**2
        new_y = y + Vy * dt + 0.5 * ay_cmd * dt**2
        new_yaw = yaw + Vyaw * dt + 0.5 * ayaw_cmd * dt**2

        # Update velocities.
        new_Vx = Vx + ax_cmd * dt
        new_Vy = Vy + ay_cmd * dt
        new_Vyaw = Vyaw + ayaw_cmd * dt

        # Update state
        self.state = np.array([new_x, new_y, new_yaw, new_Vx, new_Vy, new_Vyaw])
        # Record the frame if needed.
        if append_frame:
            self.append_frame([ax_cmd, ay_cmd, ayaw_cmd], optimal_traj, sampled_traj_list)

    def get_state(self) -> np.ndarray:
        """Return the current state."""
        return self.state.copy()

    def append_frame(self, u: np.ndarray, optimal_traj: np.ndarray, sampled_traj_list: np.ndarray) -> list:
        """
        Draw a frame of the animation.

        State vector:
            [x, y, yaw, Vx, Vy, Vyaw]
        Control input (u):
            [ax_cmd, ay_cmd, ayaw_cmd]
        """
        # Unpack the current state.
        x, y, yaw, Vx, Vy, _ = self.state
        v = np.sqrt(Vx**2 + Vy**2)  # vehicle speed

        # Unpack control inputs.
        ax_cmd, ay_cmd, ayaw_cmd = u

        ### MAIN VIEW ###
        # Draw the vehicle as a rectangle (centered at (0,0)).
        vw, vl = self.vehicle_w, self.vehicle_l
        vehicle_shape_x = [-0.5 * vl, -0.5 * vl, +0.5 * vl, +0.5 * vl, -0.5 * vl]
        vehicle_shape_y = [-0.5 * vw, +0.5 * vw, +0.5 * vw, -0.5 * vw, -0.5 * vw]
        rotated_vehicle_shape_x, rotated_vehicle_shape_y = self._affine_transform(vehicle_shape_x, vehicle_shape_y, yaw, [0, 0])
        frame = self.main_ax.plot(rotated_vehicle_shape_x, rotated_vehicle_shape_y, color='black', linewidth=2.0, zorder=3)

        # Draw the vehicle center.
        vehicle_center = patches.Circle((0, 0), radius=vw / 20.0, fc='white', ec='black', linewidth=2.0, zorder=6)
        frame += [self.main_ax.add_artist(vehicle_center)]

        # draw the reference path
        ref_path_x = self.ref_path[:, 0] - np.full(self.ref_path.shape[0], x)
        ref_path_y = self.ref_path[:, 1] - np.full(self.ref_path.shape[0], y)
        frame += self.main_ax.plot(ref_path_x, ref_path_y, color='black', linestyle="dashed", linewidth=1.5)

        ref_x, ref_y = self.ref_pos
        ref_x_rel = ref_x - x  # Adjust for vehicle position
        ref_y_rel = ref_y - y
        
        # Plot as a red star marker
        frame += self.main_ax.plot(
            [ref_x_rel], [ref_y_rel],
            marker='*', markersize=10, color='red',
            linestyle='none', label='Reference Position'
        )

        # draw the information text
        text = "vehicle velocity = {v:>+6.1f} [m/s]".format(pos_e=x, head_e=np.rad2deg(yaw), v=v)
        frame += [self.main_ax.text(0.5, 0.02, text, ha='center', transform=self.main_ax.transAxes, fontsize=14, fontfamily='monospace')]

        # Draw the predicted optimal trajectory (if any).
        if optimal_traj.any():
            opt_traj_x = np.ravel(optimal_traj[:, 0]) - x
            opt_traj_y = np.ravel(optimal_traj[:, 1]) - y
            frame += self.main_ax.plot(opt_traj_x, opt_traj_y, color='#990099', linestyle="solid",
                                        linewidth=1.5, zorder=5)

        # draw the sampled trajectories from mppi
        if sampled_traj_list.any():
            min_alpha_value = 0.25
            max_alpha_value = 0.35
            for idx, sampled_traj in enumerate(sampled_traj_list):
                # draw darker for better samples
                alpha_value = (1.0 - (idx+1)/len(sampled_traj_list)) * (max_alpha_value - min_alpha_value) + min_alpha_value
                sampled_traj_x_offset = np.ravel(sampled_traj[:, 0]) - np.full(sampled_traj.shape[0], x)
                sampled_traj_y_offset = np.ravel(sampled_traj[:, 1]) - np.full(sampled_traj.shape[0], y)
                frame += self.main_ax.plot(sampled_traj_x_offset, sampled_traj_y_offset, color='gray', linestyle="solid", linewidth=0.2, zorder=4, alpha=alpha_value)

        ### CONTROL INPUT VIEWS ###
        # Yaw acceleration view (using ayaw_cmd) on Mz_ax.
        MAX_YAW_ACCEL = self.max_ang_abs
        PIE_RATE = 3.0 / 4.0
        MAX_ACCEL = self.max_lin_abs
        PIE_STARTANGLE = 225  # degrees
        yaw_accel_abs = np.abs(ayaw_cmd)
        x_accel_abs = np.abs(ax_cmd)
        y_accel_abs = np.abs(ay_cmd)
        if ayaw_cmd < 0.0:
            yaw_pie_obj, _ = self.Mz_ax.pie(
                [MAX_YAW_ACCEL * PIE_RATE, yaw_accel_abs * PIE_RATE, (MAX_YAW_ACCEL - yaw_accel_abs) * PIE_RATE, 2 * MAX_YAW_ACCEL * (1 - PIE_RATE)],
                startangle=PIE_STARTANGLE, counterclock=False,
                colors=["lightgray", "black", "lightgray", "white"],
                wedgeprops={'linewidth': 0, "edgecolor": "white", "width": 0.4}
            )
        else:
            yaw_pie_obj, _ = self.Mz_ax.pie(
                [(MAX_YAW_ACCEL - yaw_accel_abs) * PIE_RATE, yaw_accel_abs * PIE_RATE, MAX_YAW_ACCEL * PIE_RATE, 2 * MAX_YAW_ACCEL * (1 - PIE_RATE)],
                startangle=PIE_STARTANGLE, counterclock=False,
                colors=["lightgray", "black", "lightgray", "white"],
                wedgeprops={'linewidth': 0, "edgecolor": "white", "width": 0.4}
            )
        frame += yaw_pie_obj
        frame += [self.Mz_ax.text(0, -1, f"{np.rad2deg(ayaw_cmd):+.2f} " + r"$ \rm{[deg/s^2]}$",
                                   size=14, horizontalalignment='center', verticalalignment='center', fontfamily='monospace')]

        if ax_cmd > 0.0:
            accel_pie_obj, _ = self.Fx_ax.pie(
                [MAX_ACCEL * PIE_RATE, x_accel_abs * PIE_RATE, (MAX_ACCEL - x_accel_abs) * PIE_RATE, 2 * MAX_ACCEL * (1 - PIE_RATE)],
                startangle=PIE_STARTANGLE, counterclock=False,
                colors=["lightgray", "black", "lightgray", "white"],
                wedgeprops={'linewidth': 0, "edgecolor": "white", "width": 0.4}
            )
        else:
            accel_pie_obj, _ = self.Fx_ax.pie(
                [(MAX_ACCEL - x_accel_abs) * PIE_RATE, x_accel_abs * PIE_RATE, MAX_ACCEL * PIE_RATE, 2 * MAX_ACCEL * (1 - PIE_RATE)],
                startangle=PIE_STARTANGLE, counterclock=False,
                colors=["lightgray", "black", "lightgray", "white"],
                wedgeprops={'linewidth': 0, "edgecolor": "white", "width": 0.4}
            )
        frame += accel_pie_obj
        frame += [self.Fx_ax.text(0, -1, f"{x_accel_abs:+.2f} " + r"$ \rm{[m/s^2]}$",
                                   size=14, horizontalalignment='center', verticalalignment='center', fontfamily='monospace')]

        if ay_cmd > 0.0:
            accel_pie_obj, _ = self.Fy_ax.pie(
                [MAX_ACCEL * PIE_RATE, y_accel_abs * PIE_RATE, (MAX_ACCEL - y_accel_abs) * PIE_RATE, 2 * MAX_ACCEL * (1 - PIE_RATE)],
                startangle=PIE_STARTANGLE, counterclock=False,
                colors=["lightgray", "black", "lightgray", "white"],
                wedgeprops={'linewidth': 0, "edgecolor": "white", "width": 0.4}
            )
        else:
            accel_pie_obj, _ = self.Fy_ax.pie(
                [(MAX_ACCEL - y_accel_abs) * PIE_RATE, y_accel_abs * PIE_RATE, MAX_ACCEL * PIE_RATE, 2 * MAX_ACCEL * (1 - PIE_RATE)],
                startangle=PIE_STARTANGLE, counterclock=False,
                colors=["lightgray", "black", "lightgray", "white"],
                wedgeprops={'linewidth': 0, "edgecolor": "white", "width": 0.4}
            )
        frame += accel_pie_obj
        frame += [self.Fy_ax.text(0, -1, f"{y_accel_abs:+.2f} " + r"$ \rm{[m/s^2]}$",
                                   size=14, horizontalalignment='center', verticalalignment='center', fontfamily='monospace')]

        # Append the frame.
        self.frames.append(frame)
        return frame

    def _affine_transform(self, xlist: list, ylist: list, angle: float, translation: list = [0.0, 0.0]) -> Tuple[list, list]:
        transformed_x = []
        transformed_y = []
        if len(xlist) != len(ylist):
            print("[ERROR] xlist and ylist must have the same size.")
            raise AttributeError

        for i, xval in enumerate(xlist):
            transformed_x.append(xlist[i] * np.cos(angle) - ylist[i] * np.sin(angle) + translation[0])
            transformed_y.append(xlist[i] * np.sin(angle) + ylist[i] * np.cos(angle) + translation[1])
        transformed_x.append(transformed_x[0])
        transformed_y.append(transformed_y[0])
        return transformed_x, transformed_y

    def show_animation(self, interval_ms: int) -> None:
        """Show animation of the recorded frames."""
        ani = ArtistAnimation(self.fig, self.frames, interval=interval_ms)
        html = display.HTML(ani.to_jshtml())
        display.display(html)
        plt.close()

    def save_animation(self, filename: str, interval: int, movie_writer: str = "ffmpeg") -> None:
        """Save animation of the recorded frames (ffmpeg required)."""
        ani = ArtistAnimation(self.fig, self.frames, interval=interval)
        ani.save(filename, writer=movie_writer)

# -------------------- Test Simulation -------------------- #
# sim_step = 100
# delta_t = 0.05
# # Load a reference path from CSV (assumes 2 columns: x,y)
# ref_path = np.genfromtxt('../data/ovalpath.csv', delimiter=',', skip_header=1)
# vehicle = Vehicle(ref_path=ref_path[:, 0:2])
# # Provide a 3-element control input: [ax_cmd, ay_cmd, ayaw_cmd]
# for i in range(sim_step):
#     u = [0.6 * np.sin(i / 3.0), 2.2 * np.sin(i / 10.0), 0.1 * np.cos(i / 5.0)]
#     vehicle.update(u=u, delta_t=delta_t)
# vehicle.show_animation(interval_ms=int(delta_t * 1000))


## Controller : MPPI Controller

### Note
The following MPPI implementation follows Algorithms 1 and 2 of the reference paper. 

### Reference
1. G. Williams et al. "Information-Theoretic Model Predictive Control: Theory and Applications to Autonomous Driving" 
    - URL : https://ieeexplore.ieee.org/document/8558663
    - PDF : https://arxiv.org/pdf/1707.02342.pdf

### Brief overview of MPPI algorithm
Here is a general process flow to calculate optimal input with mppi algorithm.

**[Step 1]** ramdomly sample input sequence

Mean input sequence $U$ and ramdomly sampled input sequence $V$ are defined as follows.  
Usually, optimal input sequence on the previous step is used as $U$. 
$$
    \begin{align}
        & (\mathbf{u}_0, \mathbf{u}_1, ... \mathbf{u}_{T-1}) = U \in \mathbb{R}^{m \times T}, \nonumber \\
        & (\mathbf{v}_0, \mathbf{v}_1, ... \mathbf{v}_{T-1}) = V \in \mathbb{R}^{m \times T}, \nonumber \\
        & \mathbf{v}_t = \mathbf{u}_t + \epsilon_t, \nonumber \\
        & \epsilon_t \sim \mathcal{N}(0, \Sigma).\nonumber 
    \end{align}
$$


**[Step 2]** predict future states and evaluate cost for each sample

We assume a discrete time, continuous state-action dynamical system as a control target.  
$\mathbf{x}$ is a system state, and $\mathbf{v}$ is a sampled control input.
$$
\begin{align}
\mathbf{x}_t  &\in \mathbb{R}^{n}, \nonumber \\
\mathbf{x}_{t+1} &= \mathbf{F}(\mathbf{x}_t, \mathbf{v}_t).\nonumber 
\end{align}
$$

Then costs (i.e. penalties to be minimized) for sampled sequences $S(V; \mathbf{x}_0)$ can be evaluated with following formulations.
$$
    \begin{align}
        & S(V; \mathbf{x}_0) = C(\mathcal{H}(V; \mathbf{x}_0)), \nonumber \\
        & C(\mathbf{x}_0, \mathbf{x}_1, ... \mathbf{x}_T) = \phi(\mathbf{x}_T) + \sum_{t=0}^{T-1}c(\mathbf{x}_t), \nonumber \\
        & \mathcal{H}(V; \mathbf{x}_0) = \left( \mathbf{x}_0, \mathbf{F}(\mathbf{x}_0, \mathbf{v}_0), \mathbf{F}(\mathbf{F}(\mathbf{x}_0, \mathbf{v}_0), \mathbf{v}_1), ... \right).\nonumber 
    \end{align}
$$

**[Step 3]** calculate weight for each sample sequence

Weight for a each sample sequence is derived on the basis of information theory.  
There are K sample sequences in total, represented with an index k.  
Good control sequence with small cost value get more weight, and vice versa.  

$$
\begin{align}
& w(V) = \frac{1}{\eta} \exp
\left( 
    -\frac{1}{\lambda}
    \left(
        S(V) + \lambda(1-\alpha) \sum^{T-1}_{t=0} \mathbf{u}_t^T \Sigma^{-1} (\mathbf{u}_t + \epsilon_t) - \rho
    \right)
\right) \nonumber \\
& \eta = 
\sum_{k=1}^K \exp
\left( 
    -\frac{1}{\lambda}
    \left(
        S(U + \mathcal{E}_k) + \lambda(1-\alpha) \sum^{T-1}_{t=0} \mathbf{u}_t^T \Sigma^{-1} (\mathbf{u}_t + \epsilon_t^k) - \rho
    \right)
\right)\nonumber \\
& \rho = 
\min_k 
\left( S(V_k) + \lambda(1-\alpha) \sum^{T-1}_{t=0} \mathbf{u}_t^T \Sigma^{-1} (\mathbf{u}_t + \epsilon_t^k) \right)\nonumber
\end{align}
$$

Note that $\rho$ is inserted into the formulation to avoid overflow errors during implementation.

**[Step 4]** get optimal control input sequence

Finally, optimal input trajectory for the next ($i+1$) step is given adding weighted sample sequences to the previous solution.

$$
\begin{align}
    \mathbf{u}_t^{i+1} % &= \mathbb{E}_{\mathbb{Q}_{\hat{U}, \Sigma}}[w(V)\mathbf{v}_t]
                 = u_t^i + \sum_{k=1}^K w(V_k) \epsilon_t^k \nonumber 
\end{align}
$$


In [3]:
class MPPIControllerForPathTracking():
    def __init__(
            self,
            delta_t: float = 0.05,
            max_ang_abs: float = 0.523, # [rad]
            max_lin_abs: float = 2.000, # [m/s^2]
            ref_path: np.ndarray = np.array([[0.0, 0.0, 0.0, 1.0], [10.0, 0.0, 0.0, 1.0]]),
            horizon_step_T: int = 30,
            number_of_samples_K: int = 1000,
            param_exploration: float = 0.0,
            param_lambda: float = 50.0,
            param_alpha: float = 1.0,
            sigma: np.ndarray = np.array([[0.5, 0.0, 0.0], [0.0, 0.1, 0.0]]), 
            stage_cost_weight: np.ndarray = np.array([50.0, 50.0, 1.0, 20.0]), # weight for [x, y, yaw, v]
            terminal_cost_weight: np.ndarray = np.array([50.0, 50.0, 1.0, 20.0]), # weight for [x, y, yaw, v]
            visualize_optimal_traj = True,  # if True, optimal trajectory is visualized
            visualize_sampled_trajs = False, # if True, sampled trajectories are visualized
    ) -> None:
        """initialize mppi controller for path-tracking"""
        # mppi parameters
        self.dim_x = 6 # dimension of system state vector
        self.dim_u = 3 # dimension of control input vector
        self.T = horizon_step_T # prediction horizon
        self.K = number_of_samples_K # number of sample trajectories
        self.param_exploration = param_exploration  # constant parameter of mppi
        self.param_lambda = param_lambda  # constant parameter of mppi
        self.param_alpha = param_alpha # constant parameter of mppi
        self.param_gamma = self.param_lambda * (1.0 - (self.param_alpha))  # constant parameter of mppi
        self.Sigma = sigma # deviation of noise
        self.stage_cost_weight = stage_cost_weight
        self.terminal_cost_weight = terminal_cost_weight
        self.visualize_optimal_traj = visualize_optimal_traj
        self.visualize_sampled_trajs = visualize_sampled_trajs

        # vehicle parameters
        self.delta_t = delta_t #[s]
        self.max_ang_abs = max_ang_abs # [rad]
        self.max_lin_abs = max_lin_abs # [m/s^2]
        self.ref_path = ref_path

        # mppi variables
        self.u_prev = np.zeros((self.T, self.dim_u))

        # ref_path info
        self.prev_waypoints_idx = 0
        
        self.ref_pos = (0,0)

    def calc_control_input(self, observed_x: np.ndarray) -> Tuple[float, np.ndarray]:
        """calculate optimal control input"""
        # load privious control input sequence
        u = self.u_prev

        # set initial x value from observation
        x0 = observed_x

        # get the waypoint closest to current vehicle position 
        self._get_nearest_waypoint(x0[0], x0[1], update_prev_idx=True)
        if self.prev_waypoints_idx >= self.ref_path.shape[0]-1:
            print("[ERROR] Reached the end of the reference path.")
            raise IndexError

        # prepare buffer
        S = np.zeros((self.K)) # state cost list

        # sample noise
        epsilon = self._calc_epsilon(self.Sigma, self.K, self.T, self.dim_u) # size is self.K x self.T

        # prepare buffer of sampled control input sequence
        v = np.zeros((self.K, self.T, self.dim_u)) # control input sequence with noise

        # loop for 0 ~ K-1 samples
        for k in range(self.K):         

            # set initial(t=0) state x i.e. observed state of the vehicle
            x = x0

            # loop for time step t = 1 ~ T
            for t in range(1, self.T+1):

                # get control input with noise
                if k < (1.0-self.param_exploration)*self.K:
                    v[k, t-1] = u[t-1] + epsilon[k, t-1] # sampling for exploitation
                else:
                    v[k, t-1] = epsilon[k, t-1] # sampling for exploration

                # update x
                x = self._F(x, self._g(v[k, t-1]))

                # add stage cost
                S[k] += self._c(x) + self.param_gamma * u[t-1].T @ np.linalg.inv(self.Sigma) @ v[k, t-1]

            # add terminal cost
            S[k] += self._phi(x)

        # compute information theoretic weights for each sample
        w = self._compute_weights(S)

        # calculate w_k * epsilon_k
        w_epsilon = np.zeros((self.T, self.dim_u))
        for t in range(0, self.T): # loop for time step t = 0 ~ T-1
            for k in range(self.K):
                w_epsilon[t] += w[k] * epsilon[k, t]

        # apply moving average filter for smoothing input sequence
        w_epsilon = self._moving_average_filter(xx=w_epsilon, window_size=10)

        # update control input sequence
        u += w_epsilon

        # calculate optimal trajectory
        optimal_traj = np.zeros((self.T, self.dim_x))
        if self.visualize_optimal_traj:
            x = x0
            for t in range(0, self.T): # loop for time step t = 0 ~ T-1
                x = self._F(x, self._g(u[t]))
                optimal_traj[t] = x

        # calculate sampled trajectories
        sampled_traj_list = np.zeros((self.K, self.T, self.dim_x))
        sorted_idx = np.argsort(S) # sort samples by state cost, 0th is the best sample
        if self.visualize_sampled_trajs:
            for k in sorted_idx:
                x = x0
                for t in range(0, self.T): # loop for time step t = 0 ~ T-1
                    x = self._F(x, self._g(v[k, t]))
                    sampled_traj_list[k, t] = x

        # update privious control input sequence (shift 1 step to the left)
        self.u_prev[:-1] = u[1:]
        self.u_prev[-1] = u[-1]

        # return optimal control input and input sequence
        return u[0], u, optimal_traj, sampled_traj_list, self.ref_pos

    def _calc_epsilon(self, sigma: np.ndarray, size_sample: int, size_time_step: int, size_dim_u: int) -> np.ndarray:
        """sample epsilon"""
        # check if sigma row size == sigma col size == size_dim_u and size_dim_u > 0
        if sigma.shape[0] != sigma.shape[1] or sigma.shape[0] != size_dim_u or size_dim_u < 1:
            print("[ERROR] sigma must be a square matrix with the size of size_dim_u.")
            raise ValueError

        # sample epsilon
        mu = np.zeros((size_dim_u)) # set average as a zero vector
        epsilon = np.random.multivariate_normal(mu, sigma, (size_sample, size_time_step))
        return epsilon

    def _g(self, v: np.ndarray) -> float:
        """clamp input"""
        # limit control inputs
        v[0] = np.clip(v[0], -self.max_lin_abs, self.max_lin_abs) # limit acceleraiton input
        v[1] = np.clip(v[1], -self.max_lin_abs, self.max_lin_abs) # limit acceleraiton input
        v[2] = np.clip(v[2], -self.max_ang_abs, self.max_ang_abs) # limit steering input
        return v

    def _c(self, x_t: np.ndarray) -> float:
        """Calculate stage cost with yaw rate aligned to yaw error."""
        x, y, yaw, Vx, Vy, Vyaw = x_t
        yaw = ((yaw + 2.0 * np.pi) % (2.0 * np.pi))

        # Get reference waypoint
        _, ref_x, ref_y, ref_yaw, ref_v = self._get_nearest_waypoint(x, y)

        # Compute alignment for longitudinal velocity (Vx)
        dx = ref_x - x
        dy = ref_y - y
        distance_to_ref = np.sqrt(dx**2 + dy**2)
        if distance_to_ref > 1e-3:
            alpha = np.arctan2(dy, dx)
            theta_diff = alpha - yaw
            alignment_longitudinal = np.cos(theta_diff)
            alignment_lateral = np.sin(theta_diff)
        else:
            alignment_longitudinal = 1.0
            alignment_lateral = 0.0

        # Effective speeds
        effective_speed = Vx * alignment_longitudinal
        effective_lateral_speed = Vy * alignment_lateral

        # Yaw error (normalized to [-π, π])
        yaw_error = (ref_yaw - yaw + np.pi) % (2.0 * np.pi) - np.pi

        # Yaw rate penalty: Vyaw should have the same sign as yaw_error and be proportional
        vyaw_penalty = 0.0
        if abs(yaw_error) > 1e-3:
            # Compute desired yaw rate: proportional to yaw_error with same sign
            desired_vyaw = 0.5 * yaw_error  # 1 > 0
            vyaw_penalty = (Vyaw - desired_vyaw)**2
        else:
            # No yaw error: penalize any non-zero Vyaw
            vyaw_penalty = Vyaw**2

        # Stage cost calculation
        stage_cost = (
            self.stage_cost_weight[0] * (x - ref_x)**2 + 
            self.stage_cost_weight[1] * (y - ref_y)**2 + 
            self.stage_cost_weight[2] * (yaw - ref_yaw)**2 + 
            self.stage_cost_weight[3] * (effective_speed - ref_v)**2 + 
            self.stage_cost_weight[4] * (effective_lateral_speed)**2 + 
            self.stage_cost_weight[5] * vyaw_penalty
        )
        return stage_cost

    def _phi(self, x_T: np.ndarray) -> float:
        """Calculate terminal cost with yaw rate aligned to yaw error."""
        x, y, yaw, Vx, Vy, Vyaw = x_T
        yaw = ((yaw + 2.0 * np.pi) % (2.0 * np.pi))

        # Get nearest waypoint
        _, ref_x, ref_y, ref_yaw, ref_v = self._get_nearest_waypoint(x, y)
        self.ref_pos = (ref_x, ref_y)

        # Compute alignment for longitudinal velocity (Vx)
        dx = ref_x - x
        dy = ref_y - y
        distance_to_ref = np.sqrt(dx**2 + dy**2)
        if distance_to_ref > 1e-3:
            alpha = np.arctan2(dy, dx)
            theta_diff = alpha - yaw
            alignment_longitudinal = np.cos(theta_diff)
            alignment_lateral = np.sin(theta_diff)
        else:
            alignment_longitudinal = 1.0
            alignment_lateral = 0.0

        # Effective speeds
        effective_speed = Vx * alignment_longitudinal
        effective_lateral_speed = Vy * alignment_lateral

        # Yaw error (normalized to [-π, π])
        yaw_error = (ref_yaw - yaw + np.pi) % (2.0 * np.pi) - np.pi

        # Yaw rate penalty
        vyaw_penalty = 0.0
        if abs(yaw_error) > 1e-3:
            desired_vyaw = 0.5 * yaw_error
            vyaw_penalty = (Vyaw - desired_vyaw)**2
        else:
            vyaw_penalty = Vyaw**2

        # Terminal cost calculation
        terminal_cost = (
            self.terminal_cost_weight[0] * (x - ref_x)**2 + 
            self.terminal_cost_weight[1] * (y - ref_y)**2 + 
            self.terminal_cost_weight[2] * (yaw - ref_yaw)**2 + 
            self.terminal_cost_weight[3] * (effective_speed - ref_v)**2 + 
            self.terminal_cost_weight[4] * (effective_lateral_speed)**2 + 
            self.terminal_cost_weight[5] * vyaw_penalty
        )
        return terminal_cost

    def _get_nearest_waypoint(self, x: float, y: float, update_prev_idx: bool = False):
        """search the closest waypoint to the vehicle on the reference path"""

        SEARCH_IDX_LEN = 200 # [points] forward search range
        prev_idx = self.prev_waypoints_idx
        dx = [x - ref_x for ref_x in self.ref_path[prev_idx:(prev_idx + SEARCH_IDX_LEN), 0]]
        dy = [y - ref_y for ref_y in self.ref_path[prev_idx:(prev_idx + SEARCH_IDX_LEN), 1]]
        d = [idx ** 2 + idy ** 2 for (idx, idy) in zip(dx, dy)]
        min_d = min(d)
        nearest_idx = d.index(min_d) + prev_idx

        # get reference values of the nearest waypoint
        ref_x = self.ref_path[nearest_idx,0]
        ref_y = self.ref_path[nearest_idx,1]
        ref_yaw = self.ref_path[nearest_idx,2]
        ref_v = self.ref_path[nearest_idx,3]

        # update nearest waypoint index if necessary
        if update_prev_idx:
            self.prev_waypoints_idx = nearest_idx 

        return nearest_idx, ref_x, ref_y, ref_yaw, ref_v

    def _F(self, x_t: np.ndarray, v_t: np.ndarray) -> np.ndarray:
        """calculate next state of the vehicle"""
        # get previous state variables
        x, y, yaw, Vx, Vy, Vyaw = x_t

        dt = self.delta_t

        ax_cmd = np.clip(v_t[0], -self.max_lin_abs, self.max_lin_abs)
        ay_cmd = np.clip(v_t[1], -self.max_lin_abs, self.max_lin_abs)
        ayaw_cmd = np.clip(v_t[2], -self.max_ang_abs, self.max_ang_abs)

        # Update velocities.
        new_Vx = Vx + ax_cmd * dt
        new_Vy = Vy + ay_cmd * dt
        new_Vyaw = Vyaw + ayaw_cmd * dt

        # Update positions and orientation.
        new_x = x + new_Vx * dt
        new_y = y + new_Vy * dt
        new_yaw = yaw + new_Vyaw * dt

        # Update state
        x_t_plus_1 = np.array([new_x, new_y, new_yaw, new_Vx, new_Vy, new_Vyaw])

        return x_t_plus_1

    def _compute_weights(self, S: np.ndarray) -> np.ndarray:
        """compute weights for each sample"""
        # prepare buffer
        w = np.zeros((self.K))

        # calculate rho
        rho = S.min()

        # calculate eta
        eta = 0.0
        for k in range(self.K):
            eta += np.exp( (-1.0/self.param_lambda) * (S[k]-rho) )

        # calculate weight
        for k in range(self.K):
            w[k] = (1.0 / eta) * np.exp( (-1.0/self.param_lambda) * (S[k]-rho) )
        return w

    def _moving_average_filter(self, xx: np.ndarray, window_size: int) -> np.ndarray:
        """apply moving average filter for smoothing input sequence
        Ref. https://zenn.dev/bluepost/articles/1b7b580ab54e95
        Note: The original MPPI paper uses the Savitzky-Golay Filter for smoothing control inputs.
        """
        b = np.ones(window_size)/window_size
        dim = xx.shape[1]
        xx_mean = np.zeros(xx.shape)

        for d in range(dim):
            xx_mean[:,d] = np.convolve(xx[:,d], b, mode="same")
            n_conv = math.ceil(window_size/2)
            xx_mean[0,d] *= window_size/n_conv
            for i in range(1, n_conv):
                xx_mean[i,d] *= window_size/(i+n_conv)
                xx_mean[-i,d] *= window_size/(i + n_conv - (window_size % 2)) 
        return xx_mean


## Simulation

In [ ]:
# simulation settings
delta_t = 0.05 # [sec]
sim_steps = 500 # [steps]
print(f"[INFO] delta_t : {delta_t:.2f}[s] , sim_steps : {sim_steps}[steps], total_sim_time : {delta_t*sim_steps:.2f}[s]")

# load and visualize reference path
ref_path = np.genfromtxt('../data/ovalpath.csv', delimiter=',', skip_header=1)
plt.title("Reference Path")
plt.plot(ref_path[:,0], ref_path[:,1])
plt.show()

# initialize a vehicle as a control target
vehicle = Vehicle(
    ref_path = ref_path[:, 0:2], # ndarray, size is <num_of_waypoints x 2>
)
vehicle.reset(
    init_state = np.array([0.0, 1.0, 0.0 , 0.0, 0.0, 0.0]), # [x[m], y[m], yaw[rad], v[m/s]]
)


# initialize a mppi controller for the vehicle
mppi = MPPIControllerForPathTracking(
    delta_t = delta_t*2.0, # [s]
    ref_path = ref_path, # ndarray, size is <num_of_waypoints x 2>
    horizon_step_T = 20, # [steps]
    number_of_samples_K = 500, # [samples]
    param_exploration = 0.0,
    param_lambda = 100.0,
    param_alpha = 0.98,
    sigma = np.array([[0.5, 0.5, 0.5], [0.6, 0.5, 0.5], [0.5, 0.5, 0.4]]),
    stage_cost_weight = np.array([50.0, 50.0, 50.0, 20.0, 20.0, 30.0]), # weight for [x, y, yaw, v]
    terminal_cost_weight = np.array([50.0, 50.0, 50.0, 20.0, 20.0, 30.0]), # weight for [x, y, yaw, v]
    visualize_sampled_trajs = True
)

# simulation loop
for i in range(sim_steps):

    # get current state of vehicle
    current_state = vehicle.get_state()

    try:
        # calculate input force with MPPI
        optimal_input, optimal_input_sequence, optimal_traj, sampled_traj_list, ref_pos = mppi.calc_control_input(
            observed_x = current_state
        )
    except IndexError as e:
        # the vehicle has reached the end of the reference path
        print("[ERROR] IndexError detected. Terminate simulation.")
        break

    # print current state and input force
    print(f"Time: {i*delta_t:>2.2f}[s], x={current_state[0]:>+3.3f}[m], y={current_state[1]:>+3.3f}[m], yaw={current_state[2]:>+3.3f}[rad]\n  Vx={current_state[3]:>+3.3f}[m/s], Vy={current_state[4]:>+3.3f}[m/s], , Vyaw={current_state[5]:>+3.3f}[rad/s]\n  Ax={optimal_input[0]:>+6.2f}[m/s^2], Ay={optimal_input[1]:>+6.2f}[m/s^2], , Ayaw={optimal_input[2]:>+6.2f}[m/s^2]")

    # update states of vehicle
    vehicle.update(u=optimal_input, delta_t=delta_t, optimal_traj=optimal_traj[:, 0:2], sampled_traj_list=sampled_traj_list[:, :, 0:2], ref_pos=ref_pos)

# show animation
vehicle.show_animation(interval_ms=int(delta_t * 1000))
